<a href="https://colab.research.google.com/github/satoicu/aes-rsa-crypt/blob/main/AES%26RSA_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RSA/AESを使うためpycryptodomeをインストールします

In [32]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00


これから仮のパスワードをAESを用いて暗号化、AES複合化に必要な情報をRSA暗号化します。

この処理によりRSA複合化するための公開鍵を持ったユーザーだけがAES複合化に必要な情報にアクセスできます。

まず初めに秘密鍵、公開鍵を作成します。

In [34]:
!ssh-keygen

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
/root/.ssh/id_rsa already exists.
Overwrite (y/n)? y
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:H81+BxsyuPA78YmAs+IStNlkuT8TMj1vOrDDx8iF8XQ root@57b7b1092eb5
The key's randomart image is:
+---[RSA 3072]----+
|                 |
|                 |
|     .           |
|  . = . E  +     |
| . * B oS o = o  |
|  + B O .+.+ o + |
|   + X * .++..o .|
|  . B O o o.o. . |
|   o.+.*  ..     |
+----[SHA256]-----+


秘密鍵（id_rsa）、公開鍵（id_rsa_pub）を作成しました。

パスフレーズは省略します。

パスフレーズを省略しない場合鍵情報の利用時に毎回パスフレーズの入力が必要になります。

サーバーやプログラムでの処理には向きません。

In [35]:
!cp /root/.ssh/id_rsa id_rsa
!cp /root/.ssh/id_rsa.pub id_rsa.pub
!ls

'AES&RSA_Crypto.ipynb'	 id_rsa   id_rsa.pub   sample_data   src


仮に暗号化したい文字列を「password」とします。

In [36]:
plaintext = "password"

「password」をAES暗号化します。

In [37]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

data = plaintext.encode('utf-8')

# 鍵の生成
key = get_random_bytes(32)

# AES暗号化
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
iv = cipher.nonce

ciphertext, key, tag, iv

(b'0"(\x01\xce\xe10\x0c',
 b'\xbeu\xfeK\x18x\xad]R\xde\xf6\x97Q\x04\xee\xbf\x80\x98\xdf\x82;\xa9\xb4\xbd\xd1\xe1.Xr\x01i~',
 b'{\xd6>E\xb9\xd5\x94\xb4\xd4\x9f\xfcO\xde\xd6j\\',
 b'\xf0>0\xa0W/\x06\x92\xfb\xd5\x10\xb4\xda\xa8-#')

このままではAESで暗号化したkey, tag, ivはそのままであり容易に複合化できます。

そのためkey, tag, ivをさらにRSA暗号化します。

In [132]:
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA

#  keyinfoとtag, iv, keyの結合
keyinfo = key + b'@' + tag + b'@' + iv

# RSA暗号化のため公開鍵の読み込み
with open("id_rsa.pub", "rb") as f:
    public_key = RSA.import_key(f.read())

# 暗号化
rsa = PKCS1_OAEP.new(public_key)
cipher_keyinfo = rsa.encrypt(keyinfo)

cipher_keyinfo

b'16\xa0E7]H\xa9uI\xe7\x03$\x15\xaa\xf4<\x87\xdf\xd3K\x01F\x14?\x16\xe4\xd18\xc2zk\x0cjUq\xfaV\xdc0d(\x07OK\x1a\xd3\xa2\x8b\x81\x0e\xc5\xdf\x81\x99r\xa2#\xc6)^N=\x06^\xa1\x8e\xcc\x02\x974A\xfb|c\xdf\xa1\xd09\xdf|S\x15\xf5\xdb&?\xb9t0J\xfcr\xd2\xe9\xfc\xc3\r\xda\xb0K\xed\x99\x89\xa4\xee\xb6\xbb\taF\x8e\xa5\xcb%\x9ec\x05\x8d%\x1b\x0e\xc61\xcb\xc4\xe8\x91\xe5,G1_>\xfb5,(/\xdbZ#})9XFQ\xde~\xc0\x94G\xdf/;\x06\xe5\xf49\x0b;6\xe8\xdf\xa3\x1f\xa2\x9b\x01\xc9\x99\xb3\xd2\xe4\xe9\xb4\xae\x86z\x13;\xd8\xcbY\x93\x07\xc9\xb5\x9e\xb7]3c\x0f<\x91\xa0\xb2\xfbm\xbdg\xa1\x0c\xb0Q\x0f\x86\xe2\x9ce/6\xa4\x1dG\x94W\xf1\xa9\xb9M\x96\xa08ii\xb4\xf0\xbdM\x06\n\x05\xf1\x7fID\xdf\xd7\xd0\x1cl\x18\x13\xfc\x1f>\xd2K\xb4\x19[\xeeG\x9e\xf7\xeb0u\xa41Ho5\xafN\xf0%3dVH\x91R\x99\xcd\xb0\xe1\xb7r\xbd@X\xd0\x85$`@\x05\xea6r^j\x02\xf6fn<sf\x92]\xad)\x94mGU\x0c\'\x15\x8d\x92\x0e"30\x86\xc9\xc9\x1d\xb0\xf2\xe7{\x1f@d\xae\x970\xf7\xa8\xb4v\xfd\xed\xd7\xa9eJ\xbd\xe00e\xdfc//\xc4\x03\xfa\xae\xb9]!\x0f\xd3\x93o\xcf/\xa7\x9d\x7

key, tag, ivをkeyinfoというバイト文字列にまとめRSA暗号化しました。

複合には対応する秘密鍵が必要になります。

暗号化されたcipher_keyinfoは元のkeyinfoとは異なるものであることが確認できます。

In [39]:
# 暗号化されたため元の鍵情報とは異なりFalseが返されます。
cipher_keyinfo == keyinfo

False

元々のパスワード、keyinfoを全て削除します。これにより複合を行わないと元のパスワードはわからなくなります。

In [40]:
del plaintext, key, tag, iv, keyinfo

元のplaintextを知るためにまず秘密鍵を用いてkeyinfoを複合化、その後、keyinfoを用いてplaintextを複合化します。

まず、秘密鍵を読み込みkeyinfoを複合化します。

In [41]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA

with open("id_rsa", "rb") as f:
  private_key = RSA.import_key(f.read())

# RSA復号化
decipher_rsa = PKCS1_OAEP.new(private_key)
keyinfo = decipher_rsa.decrypt(cipher_keyinfo)

# key, tag, ivを取り出す
key, tag, iv = keyinfo.split(b'@')

key, tag, iv

(b'\xbeu\xfeK\x18x\xad]R\xde\xf6\x97Q\x04\xee\xbf\x80\x98\xdf\x82;\xa9\xb4\xbd\xd1\xe1.Xr\x01i~',
 b'{\xd6>E\xb9\xd5\x94\xb4\xd4\x9f\xfcO\xde\xd6j\\',
 b'\xf0>0\xa0W/\x06\x92\xfb\xd5\x10\xb4\xda\xa8-#')

取り出した鍵情報を元にplaintextをAES複合化します。



In [42]:
# AES復号化
decipher_aes = AES.new(key, AES.MODE_EAX, iv)
decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

# 複合化されたテキストの表示
plaintext = decrypted_text.decode("utf-8")
plaintext

'password'

In [43]:
# 暗号化・複合化処理を関数化
def encrypt(plaintext, pub_key="id_rsa.pub"):
  from Crypto.Cipher import PKCS1_OAEP
  from Crypto.PublicKey import RSA
  from Crypto.Cipher import AES
  from Crypto.Random import get_random_bytes

  data = plaintext.encode('utf-8')

  # 鍵の生成
  key = get_random_bytes(32)

  # AES暗号化
  cipher = AES.new(key, AES.MODE_EAX)
  ciphertext, tag = cipher.encrypt_and_digest(data)
  iv = cipher.nonce

  #  keyinfoとtag, iv, keyの結合
  keyinfo = key + b'@' + tag + b'@' + iv

  # RSA暗号化のため公開鍵の読み込み
  with open(pub_key, "rb") as f:
      public_key = RSA.import_key(f.read())

  # 暗号化
  rsa = PKCS1_OAEP.new(public_key)
  cipherkeyinfo = rsa.encrypt(keyinfo)

  return ciphertext, cipherkeyinfo

def decrypt(ciphertext, cipherkeyinfo, sec_key="id_rsa"):
  from Crypto.Cipher import AES, PKCS1_OAEP
  from Crypto.PublicKey import RSA

  with open(sec_key, "rb") as f:
    private_key = RSA.import_key(f.read())

  # RSA復号化
  decipher_rsa = PKCS1_OAEP.new(private_key)
  keyinfo = decipher_rsa.decrypt(cipherkeyinfo)

  # key, tag, ivを取り出す
  key, tag, iv = keyinfo.split(b'@')

  # AES復号化
  decipher_aes = AES.new(key, AES.MODE_EAX, iv)
  decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

  # 複合化されたテキストの表示
  plaintext = decrypted_text.decode("utf-8")

  return plaintext

In [44]:
plaintext = "another password"
ciphertext, cipherkeyinfo = encrypt(plaintext)
decrypt(ciphertext, cipherkeyinfo)

'another password'

以上でAES、RSAを用いて特定の秘密鍵、公開鍵をやりとりした人にのみ暗号化した情報を伝えることができます。

次にプログラムで扱いやすくするためにバイト文字列をbase64を使って単なる文字列として取り扱います。

これによりjsonやyamlなどのテキストファイル形式で保存しやすくなります。
バイト文字列を単なる文字列に変換するにはbase64でエンコードした後utf-8でデコードします。

In [45]:
import base64

plaintext = "another text"
ciphertext, cipherkeyinfo = encrypt(plaintext)
b64ciphertext = base64.b64encode(ciphertext).decode('utf-8')
b64cipherkeyinfo = base64.b64encode(cipherkeyinfo).decode('utf-8')
print("base64 encoded ciphertext=", b64ciphertext)
print("base64 encoded cipherkeyinfo=", b64cipherkeyinfo)

base64 encoded ciphertext= rCTVRPdwPu7VJfKz
base64 encoded cipherkeyinfo= 2D/MomwuXX5zFVK6S/E/V/vfyoWRtYxrCIuyFCr8yXNMZr9t0VxNoVrsvLDeSt+tvVGxw51PwgwzfPnzuP558rzreygqKAm3aDPhI1LAzBuWAmyRwCd14T9Wg3aWTQPQ7jSKJxjL2S/IlWpAvgfsgmVA05FNa7V3PnRcm99zFaOqQ5IN9BF6oI+hruoIhKd4x5AvZnXI9TvNWemZzJDqLKqohGebiMrWvLJrK9qAnnbBTDKT+OxiB3jcBdFBzFc4qzArF1nnMjVQ+/3U9aZ6VxMhzEfdfyOiMGmr/ZX2Q9GK/4/tTknn2WJSz3RpQ/pAypWoEANWJo/8dVLp6PB2B0SCDkCncx4vMaOtndhJvSEZ05YCC9MbRnJ1on/M4UTsueT2CLwiBYrg1MR8VHy2XRnMbt4assXR5ZnKv0TwLdG731O95X5rI7HSP8ySfa47GHrW4QUf6btVoqi0BRX3RkoBYZWWlq6REpcYsjW0kIjYF3z2SIH377h6B8NDyHkH


utf-8のエンコードは文字列をバイト文字列に変換します。

utf-8のデコードはバイト文字列を単なる文字列に変換します。

basae64はバイト文字列を対応する64進法の英数字に当てはめて変換します。

In [46]:
b64decode_ciphertext = base64.b64decode(b64ciphertext)
b64decode_cipherkeyinfo = base64.b64decode(b64cipherkeyinfo)
decrypt(b64decode_ciphertext, b64decode_cipherkeyinfo)

'another text'

AESは非常に長い文字列も高速に暗号化できる特徴があります。

以下のような複数の情報をまとめたjson形式のファイルも暗号化できます。

サンプルはchat-gptを用いて生成した実在しない人物に対するデータの例です。

In [47]:
sample={
  "name": "John Doe",
  "age": 30,
  "email": "johndoe@example.com",
  "address": {
    "street": "123 Main St",
    "city": "New York",
    "state": "NY",
    "zip": "10001"
  },
  "phone_numbers": [
    "+1-555-1234",
    "+1-555-5678"
  ],
  "friends": [
    {
      "name": "Jane Smith",
      "age": 28,
      "email": "janesmith@example.com"
    },
    {
      "name": "Robert Johnson",
      "age": 32,
      "email": "robertjohnson@example.com"
    }
  ],
  "hobbies": [
    "reading",
    "playing guitar",
    "hiking"
  ],
  "description": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum a quam auctor, faucibus quam in, sollicitudin nisi. Nulla in urna in elit tristique interdum ac at nulla. Proin euismod scelerisque sem vitae feugiat."
}

In [48]:
import json
message = json.dumps(sample)

ciphertext, cipherkeyinfo = encrypt(message)

In [49]:
decrypted_text = decrypt(ciphertext, cipherkeyinfo)

# 結果の表示
print("暗号化前のテキスト:", message)
print("復号化後のテキスト:", decrypted_text)
print(message == decrypted_text)

暗号化前のテキスト: {"name": "John Doe", "age": 30, "email": "johndoe@example.com", "address": {"street": "123 Main St", "city": "New York", "state": "NY", "zip": "10001"}, "phone_numbers": ["+1-555-1234", "+1-555-5678"], "friends": [{"name": "Jane Smith", "age": 28, "email": "janesmith@example.com"}, {"name": "Robert Johnson", "age": 32, "email": "robertjohnson@example.com"}], "hobbies": ["reading", "playing guitar", "hiking"], "description": "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum a quam auctor, faucibus quam in, sollicitudin nisi. Nulla in urna in elit tristique interdum ac at nulla. Proin euismod scelerisque sem vitae feugiat."}
復号化後のテキスト: {"name": "John Doe", "age": 30, "email": "johndoe@example.com", "address": {"street": "123 Main St", "city": "New York", "state": "NY", "zip": "10001"}, "phone_numbers": ["+1-555-1234", "+1-555-5678"], "friends": [{"name": "Jane Smith", "age": 28, "email": "janesmith@example.com"}, {"name": "Robert Johnson", "age": 32, "email"

また、秘密鍵をスクリプトに埋め込みpyarmorライブラリでスクリプト自体を暗号化できます。

これにより秘密鍵を教えることなく複合化する機能だけを提供することができます。

In [50]:
!pip install pyarmor

pyarmorを使用するために他のスクリプトに渡しやすいようにkey, tag, ivをそれぞれbase64エンコードされた文字列として保存するようencrypt関数を更新します。

In [137]:
# 暗号化・複合化処理を関数化
def encrypt(plaintext, pub_key="id_rsa.pub"):
  from Crypto.Cipher import PKCS1_OAEP
  from Crypto.PublicKey import RSA
  from Crypto.Cipher import AES
  from Crypto.Random import get_random_bytes
  import base64

  data = plaintext.encode('utf-8')

  # 鍵の生成
  key = get_random_bytes(32)

  # AES暗号化
  cipher = AES.new(key, AES.MODE_EAX)
  ciphertext, tag = cipher.encrypt_and_digest(data)
  iv = cipher.nonce

  # RSA暗号化のため公開鍵の読み込み
  with open(pub_key, "rb") as f:
      public_key = RSA.import_key(f.read())

  # 暗号化
  rsa = PKCS1_OAEP.new(public_key)
  cipherkey = rsa.encrypt(key)
  ciphertag = rsa.encrypt(tag)
  cipheriv = rsa.encrypt(iv)

  # base64文字列化
  b64ciphertext = base64.b64encode(ciphertext).decode('utf-8')
  b64cipherkey = base64.b64encode(cipherkey).decode('utf-8')
  b64ciphertag = base64.b64encode(ciphertag).decode('utf-8')
  b64cipheriv = base64.b64encode(cipheriv).decode('utf-8')

  cipher_dict = {"text": b64ciphertext,
                 "key": b64cipherkey,
                 "tag": b64ciphertag,
                 "iv": b64cipheriv }

  return cipher_dict

pyarmorの機能を利用するために新たなパスワードを作成します。

In [153]:
plaintext = "new password for pyarmor"

plaintextを暗号化し、cipher_dictを取得します。

In [154]:
cipher_dict = encrypt(plaintext)
cipher_dict

{'text': 'aXRhXGRwxYhnDmxb1SgGPX3+ZDvNztRZ',
 'key': '2y18geJ1ZhCArGlF158DDMXAAkntUlXTNPMWKrvAHXdACLfiXZP64otxM1Or0ouubr8c7hc8PW9NkscvQQbUKBsHJePkjce+oPjxGpO5D0q+oM4mkWZEXXqzbZqTQrLlrR42rS9m4/SKiV0P2RWQELEArPimffSYA5DjVxqcu9J8cC4XT2LyTyB+suJlKGP13QlptsF2+WxUzqILgnux1MJ3BLTtnexxwTAPUCUxu0PACNDhYif21N0MzptXm3lignhzddbWq2xpIjG3QaXMuV4LP5mNMbW7DgNDwGfjEg4UsFrIthDwOml/zuRR2IoXbJgkOgr1ZrKKWqztxPSUIn++MmjSXLBWVzaQPVtj638+96prMoB4J9CMCalxWhkkwoOPyk40DVsoBa6rm4KI0rB7uRtgPkDP0LYsauy6LL0yhLWB+yTj9EY5G/dd2wyMNFeWVUDcs1RXfFddkZ9cN4Ud0o1RZVcYCD/3lENGNobJPdZxSJgVjaUpaGB98Y1C',
 'tag': 'KPrEHalIulNtvHX6RIId3z4zfk8qKHMj0kxzhFIvW/JWd5LiyG69kdodHLz1QWPmVqGb525+sYcH1XhVXol5Y7zVBoi58qrKyQh3gC1lffnB+U5z+5u3uMAzupJ56vMrzCsSRiANvGrWlR57iCBy7Jhg3QTOYfDxBvt9G5N+yEX3vNTZmEMPYyn7u6lzuQ1xXllqiJXXQdi5TfMSLDyboez4mjsrF1elqpxGBeBFroyOVy3PjeLhHWvKAQr9zfiQSUOiVoUkQkw6zAS5eK4sgISagXhsBSLCL3fQmlevwDP+EnmqvwwP9iLv/jWGR8bojD7So94a8kNFuMHZY++wyp8FKreTpuaxYI9dayxQbLwJWdz9c9+x5oaL3dklb3lQJssAkdEYplsk9yTTaYZctv/wNyAHRKMlCqrKiF

暗号化するためにスクリプトを「.py」形式で保存します。
この際スクリプト内に秘密鍵を保管します。

この状態で秘密鍵をスクリプトに含めてpyarmorを利用することで秘密鍵を教えずに複合化の機能を提供することができます。

In [155]:
with open("id_rsa", "r") as f:
  sec_key = f.read()

script = f"""def decrypt(b64ciphertext, b64cipherkey, b64ciphertag, b64cipheriv):
  from Crypto.Cipher import AES, PKCS1_OAEP
  from Crypto.PublicKey import RSA
  import base64

  ciphertext = base64.b64decode(b64ciphertext.encode('utf-8'))
  cipherkey = base64.b64decode(b64cipherkey.encode('utf-8'))
  ciphertag = base64.b64decode(b64ciphertag.encode('utf-8'))
  cipheriv = base64.b64decode(b64cipheriv.encode('utf-8'))

  sec_key = '''{sec_key}'''
  private_key = RSA.import_key(sec_key)

  # RSA復号化
  decipher_rsa = PKCS1_OAEP.new(private_key)
  key = decipher_rsa.decrypt(cipherkey)
  tag = decipher_rsa.decrypt(ciphertag)
  iv = decipher_rsa.decrypt(cipheriv)

  # AES復号化
  decipher_aes = AES.new(key, AES.MODE_EAX, iv)
  decrypted_text = decipher_aes.decrypt_and_verify(ciphertext, tag)

  # 複合化されたテキストの表示
  plaintext = decrypted_text.decode('utf-8')

  return plaintext

import sys
print("plaintext => ", decrypt(*sys.argv[1:]))
"""
open("src/decript.py", "w").write(script)

3517

pyarmorを使用して作成したdecript.pyを暗号化します。

In [156]:
!cd src && pyarmor gen decript.py

INFO     Python 3.10.12
INFO     Pyarmor 8.3.4 (trial), 000000, non-profits
INFO     Platform linux.x86_64
INFO     search inputs ...
INFO     find script decript.py
INFO     find 1 top resources
INFO     start to generate runtime files
INFO     target platforms {'linux.x86_64'}
INFO     write dist/pyarmor_runtime_000000/pyarmor_runtime.so
INFO     patch runtime file
INFO     generate runtime files OK
INFO     start to obfuscate scripts
INFO     process resource "decript"
INFO     obfuscating file decript.py
INFO     write dist/decript.py
INFO     obfuscate scripts OK


distフォルダに作成されたdecript.pyは暗号化されており中身のスクリプトを読み取ることはできません。

In [157]:
!cat src/dist/decript.py

# Pyarmor 8.3.4 (trial), 000000, non-profits, 2023-08-21T16:53:11.917616
from pyarmor_runtime_000000 import __pyarmor__
__pyarmor__(__name__, __file__, b'PY000000\x00\x03\n\x00o\r\r\n\x80\x00\x01\x00\x08\x00\x00\x00\x04\x00\x00\x00@\x00\x00\x00|\x0f\x00\x00\x12\t\x04\x00\xabN\xcc\xf5EU\xdf\xef\x12\x1d\xe5\xdf\'z}\x80\x00\x00\x00\x00\x00\x00\x00\x00\xea\x11\xc0\xe3!I\xe5\x8a\x02P\xac\xcf\x0c\x8b4c\xd91R^-/\x18\x8fx\xd6\x14\xcb\xaci6\xe5\x039x\x0e\x85zp\xbe\xd7K\xebQd\x1c\x1c\xc3@\tao\xf07\x15`\x8aX\x0e\x98\x18}\x81\xf0!>\xb0)RF\xb9\x12o\xfd\x90\xed\xa2$\x07\x87\xef\xd5\xf4f*\xb8\x01h\xe7z\xff\xe0\xba\xd8!\'\x1b\x8d\x04\xf5\x1f\x91\xac\x11\xd8\x80m\x04\xd0\x8b\xde\xb1I\x98\\n\x8bx\x84\xe1\xd4\xce\xc2\x8c\x8d\nA]q,\x05L\x1d\xd5\xc5\x7f\xc9F_\xe1\x1d\n\xe4\x9c\xd3T_|\xe5\x06GB\xbf\x8a\x84\xd8\x0e4\x11>\x1a\x93`\x95\xba\x10a7\xad\xad\xe9\x87p\xc2\x95 w\xdbcd\x04\xa7\xbf?s\x94\xcfi\\@\xc8\x07.\xd7\xcaak\xd5\xceI\x16_\x17\x14\xcd\xc21=Yx>\xf4*}\xa7\xd9&\xcd\xb3E\x8f\xfb\xae\x8f5\x14j\xbe]\x89

暗号化前のdecript.pyを削除します。

In [158]:
!rm src/decript.py

distフォルダの暗号化されたdecripty.pyを用いて元のplaintextを得ることができます

In [159]:
!python src/dist/decript.py {cipher_dict["text"]} {cipher_dict["key"]} {cipher_dict["tag"]} {cipher_dict["iv"]}

plaintext =>  new password for pyarmor
